In [1]:
import pymongo
import faker
import json
from bson import ObjectId
import hashlib

In [2]:
!pip install [biblio]

ERROR: Invalid requirement: '[biblio]'


In [3]:
# Connect to the MongoDB with user and password
db = pymongo.MongoClient("mongodb://localhost:27017/")["simulator"]

In [4]:
# Create a fake truck data generator
faker.Faker.seed(42)

fake = faker.Faker("es_ES")
trucks = db["trucks"]
#drop the collection
trucks.drop()
for i in range(20):
    truck = {
        "id": fake.license_plate_unified().replace(" ", ""),
        "capacity": fake.random_int(min=10, max=30)
    }
    trucks.insert_one(truck)

In [5]:
def create_object_id(string):
    hashed = hashlib.sha1(string.encode()).hexdigest()
    object_id = ObjectId(hashed[:24])
    return object_id

In [6]:
locations = db["locations"]
#drop the collection
locations.drop()
farmacias = json.load(open("../farmacias/farmacias.json", "r"))
#filter farmacias with no coordinates
farmacias = [farmacia for farmacia in farmacias if farmacia["coordinates"] is not None and farmacia["coordinates"]["latitude"] is not None and farmacia["coordinates"]["longitude"] is not None]
# eliminate farmacias with no name or duplicate names
farmacias = {farmacia["name"]: farmacia for farmacia in farmacias}.values()

#add _id
for farmacia in farmacias:
    farmacia["_id"] = create_object_id(farmacia["name"])

locations.insert_many(farmacias)

InsertManyResult([ObjectId('ce6feed06bc1a9e5aff95f04'), ObjectId('47700194cae2115edbe89387'), ObjectId('065e50ccf9bed218e4dc14c6'), ObjectId('e19049464a9467a9a95402ed'), ObjectId('a5f240a9777e72c31a3e69d2'), ObjectId('94a4877361dfc7ba1b87cab4'), ObjectId('15937de4acb61e0a344c6f40'), ObjectId('683d492beb28f2b6843fdc25'), ObjectId('d9cdbdb0da78a71c66b30271'), ObjectId('2c9d3210eab39a36e1f2e21b'), ObjectId('27f95bc631a2522868c52b82'), ObjectId('066eeaec0b80ec50d05f58fa'), ObjectId('b1d80a9ed76950ed2b486c8c'), ObjectId('3647bfc67b4b0b796cf02915'), ObjectId('445701b5f805c76588b9bd89'), ObjectId('80fd8dd40249feffc88145a0'), ObjectId('73ac662a164ad65152e57271'), ObjectId('8bc8213782336471754e8990'), ObjectId('eef6c322fa22198aba0c124e'), ObjectId('85ed504d7134f0484437eca6'), ObjectId('c607448f690edbe3e4451bbd'), ObjectId('58a50ddf91095c052e330fd6'), ObjectId('1793fd9ee539dc62a736ead0'), ObjectId('5247ed8a84c344dfd6061a11'), ObjectId('6912a86b7b6d344a7d91d457'), ObjectId('503420a4936ce669bcbe3a

In [7]:
# plot al the locations in a map
import folium

m = folium.Map(location=[40.416775, -3.703790], zoom_start=9)
for farmacia in farmacias:
    folium.Marker([farmacia["coordinates"]["latitude"], farmacia["coordinates"]["longitude"]], popup=farmacia["name"]).add_to(m)
m